In [79]:
# Important, 

# The aim of this code is to convert the CSV file of
# labels = missing words, and sentences with missing words
# into a tensor of numbersthat can be passed through
# the matching networks code just like the numpy array
# used for the images in the original code based on the
# Onmiglot dataset

# The numbers in the tensor will be the numbers each word
# refers to in the vocabulary we are building
# We will not embed at this stage because this is
# is done inside the matching network. We are, in effect,
# not completing the TorchText proprocessing

# This code is a mainly a mixture of two tutorials:
# http://anie.me/On-Torchtext/
# https://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/

# Comments are a mixture of those from the tutorials (most of them)
# and my own

# Note to self, use Conda environment PyTorch1

In [80]:
import pandas as pd
import numpy as np
import torch
import torchtext
from torchtext import data
from torchtext.data import Iterator
import spacy

In [81]:
# Check what the data looks like

pd.read_csv("data/train.csv").head(20)

,label,sentence
0,borgnine,hank azaria who provides the voice of apu comm...
1,borgnine,the unseen person or creature that attacks <bl...
2,borgnine,meanwhile the other junior campers led by er...
3,borgnine,<blank_token> was a guitar player in real life...
4,borgnine,seeing ned flanders get it wrong is great but ...
5,borgnine,rick porter of < unk > N it wrote in that he w...
6,borgnine,meanwhile the other junior campers led by er...
7,borgnine,ernest <blank_token> guest starred in the epis...
8,borgnine,<blank_token> apologized because he felt that ...
9,borgnine,in her book my life as a N year old boy cart...


In [82]:
# Use spacy to define a function to 
# tokenize, or split up, into individual words
# the labels and sentences Note the labels are already
# individual words

spacy_en = spacy.load('en')

def tokenizer(text): # create a tokenizer function
    return [tok.text for tok in spacy_en.tokenizer(text)]

# We first define a Field, this is a class that contains
# information on how you want the data preprocessed. It acts
# like an instruction manual that data.TabularDataset will use.
# We define two fields, one for the sentencesm and one for the
# labels

TEXT = data.Field(sequential=True, tokenize=tokenizer)
LABEL = data.Field(sequential=False, is_target=True)

In [69]:
# The fields know what to do when given raw data.
# Now, we need to tell the fields what data it
# should work on. This is where we use Datasets.

# The splits method creates a dataset for the train
# and test data by applying the same processing.

train, test = data.TabularDataset.splits(
        path='data/', train='train.csv', test='test.csv', format='csv',
        fields=[('label', LABEL), ('sentence', TEXT)])

In [ ]:
print(train)

In [71]:
# Torchtext handles mapping words to integers, but
# it has to be told the full range of words it should
# handle.

# We are currently building the vocab from the train
# and test data

TEXT.build_vocab(train, test)
LABEL.build_vocab(train, test)

# This makes torchtext go through all the elements in the
# training set, check the contents corresponding to the TEXT
# field, and register the words in its vocabulary. Torchtext
# has its own class called Vocab for handling the vocabulary.
# The Vocab class holds a mapping from word to id in its stoi
# attribute and a reverse mapping in its itos attribute.

In [ ]:
vocab = TEXT.vocab
print(vocab.Vocab)

In [73]:
# In torchvision and PyTorch, the processing and batching of
# data is handled by DataLoaders. For some reason, torchtext
# has renamed the objects that do the exact same thing to
# Iterators. The basic functionality is the same

train_iter, test_iter = Iterator.splits(
        (train, test), sort_key=lambda x: len(x.Text),
    
        # (90000, 10000) means 91270 for train and 10153 for test,
        # the number of examples in each
        # That is, we only want to create one "batch" for each
        # as we are only doing this process in TorchText to convert
        # our data into a PyTorch tensor object to be passed around
        # the matching networks program in the same way the
        # vision data was passed around in a numpy array
        # The matching networks program already takes care
        # of batching and we don;t want to distrub things too much
    
        batch_sizes=(90000, 10000)) 

In [74]:
# Look at the batch

# batch = next(train_iter.__iter__()); batch

# batch = next(iter(train_iter))
# print(batch)

In [75]:
# type(batch)

In [76]:
# Currently, the iterator returns a custom datatype
# called torchtext.data.Batch.
# we’ll convert the batch to a tuple in the form
# (x, y) where x is the label tensor
# and y is the sentence

class BatchWrapper:
    def __init__(self, dl, x_var, y_var):
        self.dl, self.x_var, self.y_var = dl, x_var, y_var # we pass in the list of attributes for x and y
    
    def __iter__(self):
        for batch in self.dl:
            x = getattr(batch, self.x_var) # we assume only one input in this wrapper
            
            y = getattr(batch, self.y_var) # we assume only one input in this wrapper

            yield (x, y)
    
    def __len__(self):
        return len(self.dl)

In [77]:
train_dl = BatchWrapper(train_iter, "label", "sentence")
test_dl = BatchWrapper(test_iter, "label", "sentence")

In [78]:
next(train_dl.__iter__())

(tensor([4023, 8962, 2049,  ..., 6208, 4272, 3103]),
 tensor([[   12,     2,    17,  ...,   496,    62,    10],
         [ 1948, 13329,    12,  ...,    27,     7,     9],
         [ 1722,     7,   192,  ...,  4017,   657,    12],
         ...,
         [    1,     1,     1,  ...,     1,     1,     1],
         [    1,     1,     1,  ...,     1,     1,     1],
         [    1,     1,     1,  ...,     1,     1,     1]]))

In [37]:
type(next(train_dl.__iter__()))

tuple

In [38]:
type(next(train_dl.__iter__())[0])

torch.Tensor

In [39]:
print(next(train_dl.__iter__())[0][0:10])

tensor([6384, 2328, 2282, 7546, 4352, 4446, 4190, 2395, 5732, 4882])


In [40]:
type(next(train_dl.__iter__())[1])

torch.Tensor

In [41]:
print(next(train_dl.__iter__())[1])

tensor([[  537,  1232,    59,  ...,     2,    29,     2],
        [   44,  2993,    62,  ..., 11267,  1354,   136],
        [  664,  5929,     7,  ...,  5018,     7,   629],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])


In [42]:
next(train_dl.__iter__())[0].shape

torch.Size([90000])

In [43]:
next(train_dl.__iter__())[1].shape

torch.Size([219, 90000])

In [47]:
# Transpose

Y_train = next(train_dl.__iter__())[0]
X_train = next(train_dl.__iter__())[1]
Y_test = next(test_dl.__iter__())[0]
X_test = next(test_dl.__iter__())[1]


X_train = X_train.t()
Y_train = Y_train.t()
X_test = X_test.t()
Y_test = Y_test.t()

RuntimeError: t() expects a 2D tensor, but self is 1D

In [5]:
# Reshape

# We convert the tensors to numpy arrays
# for the Matching Networks code so we  don't
# have to change everything that was for numpy arrays
# to PyTorch tensors. We could could then convert
# back to Tensors when we need them

# Cell incomplete, issue with size of train. Should be 90,000, not 9,000 long

X_train = X_train.reshape()
Y_train = Y_train.reshape()
X_test = X_test.reshape()
Y_test = Y_test.reshape()


In [ ]:
# Save to numpy array

np.save('X_train.npy', X_train)
np.save('Y_train.npy', Y_train)
np.save('X_test.npy', X_test)
np.save('Y_test.npy', Y_test)

tensor([[0, 1, 2],
        [3, 4, 5]])


tensor([[0, 3],
        [1, 4],
        [2, 5]])